# Modeling: Extruded plate with cut operations

As seen in the [Rectangular plate with multiple bodies](./plate_with_hole.mystnb) example,
you can create a complex sketch with holes and extrude it to create a body. However, you can also
perform cut operations on the extruded body to achieve similar results.

## Perform required imports

Perform the required imports.

In [1]:
from pint import Quantity

from ansys.geometry.core import launch_modeler
from ansys.geometry.core.math import Plane, Point3D, Point2D
from ansys.geometry.core.misc import UNITS
from ansys.geometry.core.sketch import Sketch

## Define sketch profile without holes

Create a sketch profile for the proposed design. The sketch is the same as the
[Rectangular plate with multiple bodies](./plate_with_hole.mystnb) example, but without the holes.

These holes are created by performing cut operations on the extruded body in the next steps.

In [2]:
sketch = Sketch()
(sketch.segment(Point2D([-4, 5], unit=UNITS.m), Point2D([4, 5], unit=UNITS.m))
    .segment_to_point(Point2D([4, -5], unit=UNITS.m))
    .segment_to_point(Point2D([-4, -5], unit=UNITS.m))
    .segment_to_point(Point2D([-4, 5], unit=UNITS.m))
    .box(Point2D([0,0], unit=UNITS.m), Quantity(3, UNITS.m), Quantity(3, UNITS.m))
)

modeler = launch_modeler()
design = modeler.create_design("ExtrudedPlateNoHoles")
body = design.extrude_sketch(f"PlateLayer", sketch, Quantity(2, UNITS.m))

design.plot()

/home/runner/work/pyansys-geometry/pyansys-geometry/.venv/lib/python3.13/site-packages/ansys/tools/common/cyberchannel.py:187: UserWarning: Starting gRPC client without TLS on localhost:700. This is INSECURE. Consider using a secure connection.
  warn(f"Starting gRPC client without TLS on {target}. This is INSECURE. Consider using a secure connection.")


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

## Define sketch profile for holes

Create a sketch profile for the holes in the proposed design. The holes are created by
sketching circles at the four corners of the plate. First create a reference sketch
for all the circles. This sketch is translated to the four corners of the plate.

In [3]:
sketch_hole = Sketch()
sketch_hole.circle(Point2D([0, 0], unit=UNITS.m), Quantity(0.5, UNITS.m))

hole_centers = [
    Plane(Point3D([3, 4, 0], unit=UNITS.m)),
    Plane(Point3D([-3, 4, 0], unit=UNITS.m)),
    Plane(Point3D([-3, -4, 0], unit=UNITS.m)),
    Plane(Point3D([3, -4, 0], unit=UNITS.m)),
]

## Perform cut operations on the extruded body

Perform cut operations on the extruded body to create holes at the four corners of the plate.

In [4]:
for center in hole_centers:
    sketch_hole.plane = center
    design.extrude_sketch(
        name= f"H_{center.origin.x}_{center.origin.y}",
        sketch=sketch_hole,
        distance=Quantity(2, UNITS.m),
        cut=True,
    )

design.plot()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

## Close the modeler

Close the modeler to free up resources and release the connection.

In [5]:
modeler.close()